In [10]:
import torch

In [11]:
def quantize_to_element_format(value, max_element_value):
    """Квантование значения в представимый диапазон формата элемента."""
    if value > max_element_value:
        return max_element_value
    elif value < -max_element_value:
        return -max_element_value
    else:
        return value

def convert_to_mx_format(tensor):
    """Преобразует тензор скалярных чисел в формат MX."""
    emax_elem = 127  # Для FP32, максимальная экспонента нормального числа
    max_float32 = 3.4028235e38  # Максимальное представимое значение FP32

    # Проверка на бесконечные и NaN значения
    if torch.any(torch.isinf(tensor)) or torch.any(torch.isnan(tensor)):
        raise ValueError("Входной тензор содержит бесконечные или NaN значения")

    # Вычисляем максимальное значение по модулю тензора
    max_abs_value = torch.max(torch.abs(tensor))

    # Если все элементы равны 0
    if max_abs_value.item() == 0:
        return 0, torch.zeros(tensor.size())

    # 1. Вычисляем общий масштаб
    shared_exp = int(torch.floor(torch.log2(max_abs_value))) - emax_elem
    X = 2 ** shared_exp

    # 2. Квантование элементов
    P = []
    for value in tensor:
        # Нормируем значение
        quantized_value = quantize_to_element_format(value / X, max_float32)
        P.append(quantized_value)

    return X, torch.tensor(P)

In [ ]:
input_tensor = torch.tensor([1.0, 2.5, 3.0, -4.5, 0.0, 1.0e-10, 1.7], dtype=torch.float32)
try:
    X, P = convert_to_mx_format(input_tensor)
    print("Общий масштаб X:", X)
    print("Квантованные элементы P:", P)
except ValueError as e:
    print(e)

In [ ]:
import torch

# Настройки
M, K, N = 4, 3, 2  # Размерности для матриц
bfloat16 = torch.bfloat16  # Используем BFloat16

# Инициализация входных данных и весов
A_i_1 = torch.randn(M, K, dtype=torch.float32).to(bfloat16)  # Вход предыдущего слоя (BFloat16)
W_i = torch.randn(K, N, dtype=torch.float32).to(bfloat16)  # Веса текущего слоя (BFloat16)

# Прямой проход
def forward_pass(A_i_1, W_i):
    # Квантизация (в данном случае просто преобразуем в BFloat16)
    A_i_1_quantized = A_i_1.to(bfloat16)
    W_i_quantized = W_i.to(bfloat16)

    # Матрица A_i (выход текущего слоя)
    A_i = torch.matmul(A_i_1_quantized, W_i_quantized)

    return A_i

# Обратный проход
def backward_pass(A_i_1, W_i, grad_output):
    # Квантизация градиента (если нужно)
    grad_output_quantized = grad_output.to(bfloat16)

    # Вычисление градиентов
    G_i = torch.matmul(A_i_1.T, grad_output_quantized)  # Градиенты по весам
    grad_A_i_1 = torch.matmul(grad_output_quantized, W_i.T)  # Градиенты по A_i_1

    return G_i, grad_A_i_1

# Основной код
A_i = forward_pass(A_i_1, W_i)
print("A_i (выход текущего слоя):")
print(A_i)

# Предположим, что у нас есть градиенты потерь по выходу
grad_output = torch.randn(M, N, dtype=torch.float32)

G_i, grad_A_i_1 = backward_pass(A_i_1, W_i, grad_output)
print("\nG_i (градиенты по весам):")
print(G_i)

print("\nГрадиенты по A_i_1:")
print(grad_A_i_1)


In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Настройки
M, K, N = 4, 3, 2  # Размерности для матриц
bfloat16 = torch.bfloat16  # Используем BFloat16

# Определяем двуслойную нейронную сеть
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(K, N).to(bfloat16)  # Первый слой (вход K, выход N) в bfloat16
        self.fc2 = nn.Linear(N, N).to(bfloat16)  # Второй слой (вход N, выход N) в bfloat16

    def forward(self, x):
        x = self.fc1(x)  # Прямой проход через первый слой
        x = F.relu(x)    # Применяем ReLU
        x = self.fc2(x)  # Прямой проход через второй слой
        return x

# Инициализация сети
model = SimpleNN()

# Инициализация входных данных
A_i_1 = torch.randn(M, K, dtype=torch.bfloat16)  # Входные данные (BFloat16)

# Прямой проход
def forward_pass(model, A_i_1):
    model.eval()  # Устанавливаем режим оценки
    with torch.no_grad():
        A_i = model(A_i_1)  # Проходим через сеть
    return A_i

# Обратный проход
def backward_pass(model, A_i_1, grad_output):
    model.train()  # Устанавливаем режим обучения
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)  # Оптимизатор

    # Прямой проход для получения предсказания
    A_i = model(A_i_1)
    A_i = A_i.cuda()
    grad_output = grad_output.cuda()

    # Вычисляем функцию потерь (например, среднеквадратичную ошибку)
    loss = F.mse_loss(A_i, grad_output)
    
    # Обнуляем градиенты
    optimizer.zero_grad()

    # Обратный проход
    loss.backward()
    optimizer.step()

    return loss.item()

# Основной код
# Получаем выход нейронной сети
A_i = forward_pass(model, A_i_1)
print("A_i (выход текущего слоя):")
print(A_i)

# Предположим, что у нас есть целевые градиенты (град. выход)
grad_output = torch.randn(M, N, dtype=torch.bfloat16)  # Используем BFloat16 для целевых градиентов

# Запускаем обратный проход и получаем значение потерь
loss_value = backward_pass(model, A_i_1, grad_output)
print("\nЗначение потерь:")
print(loss_value)


A_i (выход текущего слоя):
tensor([[-0.0391, -0.0972],
        [ 0.1465, -0.1099],
        [-0.0461, -0.1348],
        [ 0.3926, -0.0669]], dtype=torch.bfloat16)

Значение потерь:
2.65625
